## Device Setup

In [1]:
!nvidia-smi

Tue Dec 23 05:15:29 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   58C    P0              71W / 300W |    850MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
print(f"Current PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")

Current PyTorch version: 2.9.1+cu128
CUDA available: True
CUDA version: 12.8


## Passage dataset

***
Percent of Spammers: 10, 20, 40, 60, 80

10 random seeds: range(20, 30)
for data creation for each type of spammer

invoke factor barp

get barp accuracy (+- std dev), wacc and tau

save in results/spammer_type/barp.csv
***

In [4]:
import sys
sys.path.insert(0, "../")
sys.path.insert(1, "../../")

from spammer_types import *
from util import *
import opt_fair
from distribution_utils import crowd_bt_dist, logistic_preference_dist, comparisons_to_df, safe_kendalltau, to_numpy
from metrics import compute_acc, compute_weighted_acc

### Get the original df of passage dataset

In [5]:
df_path = "../../real_data/faceage/data/crowd_labels.csv"

In [6]:
import pandas as pd
df = pd.read_csv(df_path)
def sort_df(df, column_name):
        # Sort by a specific column (replace 'column_name' with your column)
        df_sorted = df.sort_values(by=column_name, ascending=True)  # or ascending=False

        return df_sorted
df = sort_df(df, 'performer')
df[['left', 'right', 'label', 'performer']].head()

,left,right,label,performer
0,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,0
6306,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,0
6176,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,0
6175,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,0
6174,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,0


In [7]:
percents = [10, 20, 40, 60, 80]
# percents = [10]

In [8]:
import pickle

with open("../../real_data/faceage/data/FaceAgeDF.pickle", "rb") as handle:
    df_passage = pickle.load(handle)
df_passage

,full_path,score,gender
0,nm1442940_rm3965098752_1996-10-3_2006.jpg,10,0.0
1,nm4832920_rm1781768448_2003-8-28_2013.jpg,10,0.0
2,nm0652089_rm860657920_1992-3-10_2002.jpg,10,0.0
3,nm0004917_rm1493730304_1969-5-12_1979.jpg,10,0.0
4,nm1113550_rm1332711936_1996-4-14_2006.jpg,10,0.0
...,...,...,...
9145,475367_1941-08-03_2011.jpg,70,1.0
9146,304085_1919-07-07_1989.jpg,70,1.0
9147,nm0001627_rm4164078592_1927-2-20_1997.jpg,70,1.0
9148,nm0000024_rm1715129344_1904-4-14_1974.jpg,70,1.0


In [9]:
size = len(df_passage)
print(size)
classes = [0] * size
# for faceage it would be classes = df_passage['gender']

9150


In [10]:
gt_df = df_passage

### Addition of Random Guessors

In [11]:
spammer_type = "random"

In [12]:
csv_file = f"results/{spammer_type}/barp.csv"

In [13]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [14]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "BARP_acc_mean", "BARP_acc_std",
    "BARP_wacc_mean", "BARP_wacc_std",
    "BARP_tau_mean", "BARP_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
for percent in percents:
    # initialize metrics
    BARP_accs, BARP_waccs, BARP_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_random_spammer(df, percent, seed=sd)
        PC_faceage = df_to_pickle_faceage(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        
        try:
            FaceAge = opt_fair.BARP(data=PC_faceage, penalty=0, classes=classes, device=device)
            annot_bt_temp, annot_bias = opt_fair._alternate_optim_torch(size, K, FaceAge, iters=100)
            annot_bt_np = to_numpy(annot_bt_temp)
            if np.isnan(annot_bt_np).any():
                continue
            BARP_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
            if BARP_tau < 0:
                annot_bt_np = -annot_bt_np
            BARP_acc = compute_acc(gt_df, annot_bt_np, device)
            BARP_wacc = compute_weighted_acc(gt_df, annot_bt_np, device)
            BARP_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
        except Exception as e:
            print(f"BARP failed due to {e}")
            continue
        BARP_accs.append(BARP_acc)
        BARP_waccs.append(BARP_wacc)
        BARP_taus.append(BARP_tau)
    
    row = [
        percent,
        np.mean(BARP_accs), np.std(BARP_accs),
        np.mean(BARP_waccs), np.std(BARP_waccs),
        np.mean(BARP_taus), np.std(BARP_taus)
    ]
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"BARP | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(BARP_accs):.4f} ± {np.std(BARP_accs):.4f} | "
    f"WAcc: {np.mean(BARP_waccs):.4f} ± {np.std(BARP_waccs):.4f} | "
    f"Tau: {np.mean(BARP_taus):.4f} ± {np.std(BARP_taus):.4f}")

Unique performers: 4500
4500


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [08:05<00:00,  4.85s/it]


Unique performers: 4500
4500


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [07:46<00:00,  4.66s/it]


Unique performers: 4500
4500


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [08:20<00:00,  5.01s/it]


Unique performers: 4909
4909


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [08:34<00:00,  5.15s/it]


Unique performers: 4909
4909


 75%|████████████████████████████████████████████████████████████████████████████████████████▌                             | 75/100 [06:19<02:14,  5.38s/it]

### Addition of Anti-Personas

In [11]:
spammer_type = "anti"

In [12]:
csv_file = f"results/{spammer_type}/barp.csv"

In [13]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [14]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "BARP_acc_mean", "BARP_acc_std",
    "BARP_wacc_mean", "BARP_wacc_std",
    "BARP_tau_mean", "BARP_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
for percent in percents:
    # initialize metrics
    BARP_accs, BARP_waccs, BARP_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_anti_personas(df, percent, seed=sd)
        PC_faceage = df_to_pickle_faceage(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        
        try:
            FaceAge = opt_fair.BARP(data=PC_faceage, penalty=0, classes=classes, device=device)
            annot_bt_temp, annot_bias = opt_fair._alternate_optim_torch(size, K, FaceAge, iters=100)
            annot_bt_np = to_numpy(annot_bt_temp)
            if np.isnan(annot_bt_np).any():
                continue
            BARP_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
            if BARP_tau < 0:
                annot_bt_np = -annot_bt_np
            BARP_acc = compute_acc(gt_df, annot_bt_np, device)
            BARP_wacc = compute_weighted_acc(gt_df, annot_bt_np, device)
            BARP_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
        except Exception as e:
            print(f"BARP failed due to {e}")
            continue
        BARP_accs.append(BARP_acc)
        BARP_waccs.append(BARP_wacc)
        BARP_taus.append(BARP_tau)
    
    row = [
        percent,
        np.mean(BARP_accs), np.std(BARP_accs),
        np.mean(BARP_waccs), np.std(BARP_waccs),
        np.mean(BARP_taus), np.std(BARP_taus)
    ]
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"BARP | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(BARP_accs):.4f} ± {np.std(BARP_accs):.4f} | "
    f"WAcc: {np.mean(BARP_waccs):.4f} ± {np.std(BARP_waccs):.4f} | "
    f"Tau: {np.mean(BARP_taus):.4f} ± {np.std(BARP_taus):.4f}")

Unique performers: 4500
4500


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [05:13<00:00,  3.14s/it]


Unique performers: 4500
4500


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [05:00<00:00,  3.00s/it]


Unique performers: 4500
4500


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [05:00<00:00,  3.01s/it]


Unique performers: 4500
4500


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [04:59<00:00,  2.99s/it]


BARP | Percent: 10 |Acc: 0.7855 ± 0.0000 | WAcc: 0.8681 ± 0.0000 | Tau: 0.5664 ± 0.0001
Unique performers: 4909
4909


 32%|█████████████████████████████████████▊                                                                                | 32/100 [01:44<03:41,  3.26s/it]

### Addition of Left Position Biased Spammers

In [ ]:
spammer_type = "left"

In [ ]:
csv_file = f"results/{spammer_type}/barp.csv"

In [ ]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [ ]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "BARP_acc_mean", "BARP_acc_std",
    "BARP_wacc_mean", "BARP_wacc_std",
    "BARP_tau_mean", "BARP_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
for percent in percents:
    # initialize metrics
    BARP_accs, BARP_waccs, BARP_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_position_biased_spammers(df, percent, position_bias="left", seed=sd)
        PC_faceage = df_to_pickle_faceage(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        
        try:
            FaceAge = opt_fair.BARP(data=PC_faceage, penalty=0, classes=classes, device=device)
            annot_bt_temp, annot_bias = opt_fair._alternate_optim_torch(size, K, FaceAge, iters=100)
            annot_bt_np = to_numpy(annot_bt_temp)
            if np.isnan(annot_bt_np).any():
                continue
            BARP_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
            if BARP_tau < 0:
                annot_bt_np = -annot_bt_np
            BARP_acc = compute_acc(gt_df, annot_bt_np, device)
            BARP_wacc = compute_weighted_acc(gt_df, annot_bt_np, device)
            BARP_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
        except Exception as e:
            print(f"BARP failed due to {e}")
            continue
        BARP_accs.append(BARP_acc)
        BARP_waccs.append(BARP_wacc)
        BARP_taus.append(BARP_tau)
    
    row = [
        percent,
        np.mean(BARP_accs), np.std(BARP_accs),
        np.mean(BARP_waccs), np.std(BARP_waccs),
        np.mean(BARP_taus), np.std(BARP_taus)
    ]
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"BARP | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(BARP_accs):.4f} ± {np.std(BARP_accs):.4f} | "
    f"WAcc: {np.mean(BARP_waccs):.4f} ± {np.std(BARP_waccs):.4f} | "
    f"Tau: {np.mean(BARP_taus):.4f} ± {np.std(BARP_taus):.4f}")

### Addition of Right Position Biased Spammers

In [ ]:
spammer_type = "right"

In [ ]:
csv_file = f"results/{spammer_type}/barp.csv"

In [ ]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [ ]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "BARP_acc_mean", "BARP_acc_std",
    "BARP_wacc_mean", "BARP_wacc_std",
    "BARP_tau_mean", "BARP_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
for percent in percents:
    # initialize metrics
    BARP_accs, BARP_waccs, BARP_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_position_biased_spammers(df, percent, position_bias="right", seed=sd)
        PC_faceage = df_to_pickle_faceage(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        
        try:
            FaceAge = opt_fair.BARP(data=PC_faceage, penalty=0, classes=classes, device=device)
            annot_bt_temp, annot_bias = opt_fair._alternate_optim_torch(size, K, FaceAge, iters=100)
            annot_bt_np = to_numpy(annot_bt_temp)
            if np.isnan(annot_bt_np).any():
                continue
            BARP_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
            if BARP_tau < 0:
                annot_bt_np = -annot_bt_np
            BARP_acc = compute_acc(gt_df, annot_bt_np, device)
            BARP_wacc = compute_weighted_acc(gt_df, annot_bt_np, device)
            BARP_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
        except Exception as e:
            print(f"BARP failed due to {e}")
            continue
        BARP_accs.append(BARP_acc)
        BARP_waccs.append(BARP_wacc)
        BARP_taus.append(BARP_tau)
    
    row = [
        percent,
        np.mean(BARP_accs), np.std(BARP_accs),
        np.mean(BARP_waccs), np.std(BARP_waccs),
        np.mean(BARP_taus), np.std(BARP_taus)
    ]
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"BARP | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(BARP_accs):.4f} ± {np.std(BARP_accs):.4f} | "
    f"WAcc: {np.mean(BARP_waccs):.4f} ± {np.std(BARP_waccs):.4f} | "
    f"Tau: {np.mean(BARP_taus):.4f} ± {np.std(BARP_taus):.4f}")

### Addition of Equal Proportion of all four types of spammers

In [11]:
spammer_type = "equal"

In [12]:
csv_file = f"results/{spammer_type}/barp.csv"

In [13]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [14]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "BARP_acc_mean", "BARP_acc_std",
    "BARP_wacc_mean", "BARP_wacc_std",
    "BARP_tau_mean", "BARP_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
for percent in percents:
    # initialize metrics
    BARP_accs, BARP_waccs, BARP_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_equal_proportion_of_all_spammers(df, percent, seed=sd)
        PC_faceage = df_to_pickle_faceage(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        
        try:
            FaceAge = opt_fair.BARP(data=PC_faceage, penalty=0, classes=classes, device=device)
            annot_bt_temp, annot_bias = opt_fair._alternate_optim_torch(size, K, FaceAge, iters=100)
            annot_bt_np = to_numpy(annot_bt_temp)
            if np.isnan(annot_bt_np).any():
                continue
            BARP_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
            if BARP_tau < 0:
                annot_bt_np = -annot_bt_np
            BARP_acc = compute_acc(gt_df, annot_bt_np, device)
            BARP_wacc = compute_weighted_acc(gt_df, annot_bt_np, device)
            BARP_tau = safe_kendalltau(annot_bt_np, gt_df['score'].to_numpy())
        except Exception as e:
            print(f"BARP failed due to {e}")
            continue
        BARP_accs.append(BARP_acc)
        BARP_waccs.append(BARP_wacc)
        BARP_taus.append(BARP_tau)
    
    row = [
        percent,
        np.mean(BARP_accs), np.std(BARP_accs),
        np.mean(BARP_waccs), np.std(BARP_waccs),
        np.mean(BARP_taus), np.std(BARP_taus)
    ]
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"BARP | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(BARP_accs):.4f} ± {np.std(BARP_accs):.4f} | "
    f"WAcc: {np.mean(BARP_waccs):.4f} ± {np.std(BARP_waccs):.4f} | "
    f"Tau: {np.mean(BARP_taus):.4f} ± {np.std(BARP_taus):.4f}")

Unique performers: 4500
4500


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [05:45<00:00,  3.46s/it]


Unique performers: 4500
4500


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [05:37<00:00,  3.38s/it]


Unique performers: 4500
4500


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [06:21<00:00,  3.82s/it]


Unique performers: 4909
4909


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [06:02<00:00,  3.62s/it]


Unique performers: 4909
4909


 93%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████▋        | 93/100 [05:42<00:26,  3.72s/it]